In [3]:
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from sage.all import *
import math
import secrets
from collections import deque

In [ ]:
p = 13
Fp = GF(p)
g = 4
print(g.multiplicative_order())
print(p, g)

+Infinity
13 4


In [1]:
51%7

2

In [ ]:
p = ZZ(7)
Fp = GF(p)
g = ZZ(2)
print(g.multiplicative_order())
print(p, g)

TypeError: multiplicative_order() takes exactly 0 positional arguments (1 given)

In [48]:
import numpy as np
y_ = np.array([2,2,2,4,2,1,1])
u_= np.array([0,3,0,5,1,0,2])
e_=np.array([4,2,1,4,2,1,1])
y = (u_+2*e_%7)%7
print(y)
print(np.multiply(u_,y)%7)
y_new = np.multiply(y,y_)%7
print(y_new)

[1 0 2 6 5 2 4]
[0 0 0 2 5 0 1]
[2 0 4 3 3 2 4]


In [3]:
H = np.array([[1,0,0,4,2,2,3], 
 [0,1,0,3,5,6,5],
 [0,0,1,5,5,3,4]])
e = np.array([1,4,2,2,4,1,1])
s = np.matmul(e,H.transpose())%7
print(s)
left_term = np.matmul(y_new,H.transpose())%7
print(2*s%7)
print((left_term - 2*s)%7)
np.linalg.inv(H)*s%7

[1 6 4]
[2 5 1]
[6 2 6]


LinAlgError: Last 2 dimensions of the array must be square

In [ ]:
import hashlib

p = ZZ(7)
Fp = GF(p)

def expand_seed_to_Group(seed: bytes, group:str, p: int = 7, n: int = 7, E: list = [1, 2, 4]):
    # Expandir la semilla con SHAKE128 para generar n bytes adicionales
    shake = hashlib.shake_128()
    shake.update(seed)
    
    # Generar suficientes bytes (uno por elemento del vector)
    expanded_bytes = shake.digest(n)  # Genera 'n' bytes
    
    # Convertir bytes a enteros y mapear a Fp
    if group == "Fp":
        vector_f7 = vector(GF(p), [b % p for b in expanded_bytes])
        return vector_f7
    #elif group == "b":
    #    b = vector([E[b % len(E)] for b in expanded_bytes])
    #    return vector_f7
    else: 
        vector_E = vector([E[b % len(E)] for b in expanded_bytes])
        return vector_E

# Ejemplo de uso con una semilla de 384 bits (48 bytes)
seed = bytes.fromhex("a3b1c9d4e7f2a0b5c6d8e9f1a2b3c4d5e6f7890123456789abcdef0123456789")
salt = "f0ffa34335ef979fcf79a200874749da3054fe398bb6d2137d3c98b82df9160f"
seeds = ["7170d6c7f8b636d4792500fcdddd08cd", "1f89eb454f9883c039e15ba36f444889", "0a9cbd508c727b2588ef3d9ead2f4af6"]
print('Semillas del árbol de semillas\n')
i = 1
while i <= len(seeds):
    seed_i = seeds[i-1]
    seed = seed_i+salt+'f'*15+str(i)
    print(seed)

    i += 1


seeds_ue = ["45f0253de425cf88b474ded11b1bedd64cf45dbb79d192feaa3ac88a5f0868ad", "510c963112055f48645d4a467c84d00660e1c04e10f328bfb338eabe4ae32658", "b85818685f41eca4090f44d4b4a6037990393ecf89b495c9ff4ceeac8090926e"]
print("\nSemillas Seedu' y Seed e'\n")
i = 0

seeds_str = "["
ue_str = "["
u_vectors = []
e_vectors = []
for seed_ue in seeds_ue:
    seed_u = seeds_ue[i][:32]
    seeds_str += str(seed_u) + "|"
    seed_u = bytes.fromhex(seed_u)
    seed_e = seeds_ue[i][32:]
    seeds_str += str(seed_e) +"; "
    seed_e = bytes.fromhex(seed_e)
    vector_f7 = expand_seed_to_Group(seed_u, 'Fp')
    u_vectors.append(vector_f7)
    e_vectors.append(vector_e)
    vector_e = expand_seed_to_Group(seed_e, 'E')
    ue_str += "u':" + str(vector_f7) + ", e':"+ str(vector_e) +"; "
    print(f"{i+1}. Seedu': {seeds_ue[i][:32]}, Seede': {seeds_ue[i][32:]}\nResultados:\nu': {vector_f7}, e': {vector_e}")

    i += 1
seeds_str = seeds_str[:-2] + "]"
ue_str = ue_str[:-2] + "]"
print(seeds_str)
print(ue_str)

Semillas del árbol de semillas

7170d6c7f8b636d4792500fcdddd08cdf0ffa34335ef979fcf79a200874749da3054fe398bb6d2137d3c98b82df9160ffffffffffffffff1
1f89eb454f9883c039e15ba36f444889f0ffa34335ef979fcf79a200874749da3054fe398bb6d2137d3c98b82df9160ffffffffffffffff2
0a9cbd508c727b2588ef3d9ead2f4af6f0ffa34335ef979fcf79a200874749da3054fe398bb6d2137d3c98b82df9160ffffffffffffffff3

Semillas Seedu' y Seed e'

1. Seedu': 45f0253de425cf88b474ded11b1bedd6, Seede': 4cf45dbb79d192feaa3ac88a5f0868ad
Resultados:
u': (6, 0, 3, 4, 2, 6, 2), e': (4, 2, 4, 1, 2, 4, 1)
2. Seedu': 510c963112055f48645d4a467c84d006, Seede': 60e1c04e10f328bfb338eabe4ae32658
Resultados:
u': (5, 2, 2, 2, 1, 5, 4), e': (4, 2, 2, 1, 1, 4, 1)
3. Seedu': b85818685f41eca4090f44d4b4a60379, Seede': 90393ecf89b495c9ff4ceeac8090926e
Resultados:
u': (6, 0, 0, 3, 5, 5, 5), e': (1, 2, 2, 1, 2, 1, 4)
[45f0253de425cf88b474ded11b1bedd6|4cf45dbb79d192feaa3ac88a5f0868ad; 510c963112055f48645d4a467c84d006|60e1c04e10f328bfb338eabe4ae32658; b85818685f41e

In [112]:
import hashlib

H = np.array([[1,0,0,4,2,2,3], 
 [0,1,0,3,5,6,5],
 [0,0,1,5,5,3,4]])
e = np.array([1,4,2,2,4,1,1])
s = np.matmul(e,H.transpose())%7
Fz = [0, 1, 2]
g = 4

#Cálculo de la función sigma
sigma_ops = []
sigmas = []
direct_mapping = {1:0, 4:1, 2:2}
for e_vector in e_vectors:
    print(e, e_vector)
    e_z, e_vector_z = vector([direct_mapping[value] for value in e]), vector([direct_mapping[value] for value in e_vector])
    print(e_z, e_vector_z)
    substract = (e_z - e_vector_z)
    y_z = vector([Fz[b % len(Fz)] for b in substract])
    sigmas.append(y_z)
    y = vector([g**(power) for power in y_z])%7
    print(y) #sigma(v) equiv. y*v, donde * es producto componente a componente
    sigma_ops.append(y)

#sindrome calculation
sindromes = []
i = 0
for u_vector in u_vectors:
    u = vector([u_vector[j]*sigma_ops[i][j] for j in range(len(u_vector))]) #vector u
    s_i = vector(np.matmul(u,H.transpose())%7)
    sindromes.append(s_i)
    i += 1
print(sindromes)

#cmt0 and cmt1 calculation
cmt0 = []
cmt1 = []
for i in range(len(sindromes)):
    #cmt0
    s_bytes = bytes([int(x) for x in sindromes[i]])
    sigma_bytes = bytes([int(x) for x in sigmas[i]])

    # Convertir salt desde hex a bytes
    salt_bytes = bytes.fromhex(salt)

    # Convertir i a 8 bytes (64 bits, big-endian)
    i_bytes = i.to_bytes(8, 'big')

    # Concatenar todos los valores
    concatenated = s_bytes + sigma_bytes + salt_bytes + i_bytes
    print(s_bytes, sigma_bytes, salt_bytes, i_bytes)
    print(concatenated)

    # Calcular SHA3-256
    cmt0_i = hashlib.sha3_256(concatenated).hexdigest()

    #cmt1
    seed_tree_i = bytes.fromhex(seeds[i]) + salt_bytes + (i+1).to_bytes(8, 'big')
    
     # Concatenar todos los valores
    concatenated = seed_tree_i + salt_bytes + i_bytes
    print(seed_tree_i, salt_bytes, i_bytes)
    print(concatenated)

    # Calcular SHA3-256
    cmt1_i = hashlib.sha3_256(concatenated).hexdigest()
    
    cmt0.append(cmt0_i)
    cmt1.append(cmt1_i)
    print(f'cmt0[{i}]:',cmt0,f'cmt1[{i}]:',cmt1)

[1 4 2 2 4 1 1] (1, 2, 2, 1, 2, 1, 4)
(0, 1, 2, 2, 1, 0, 0) (0, 2, 2, 0, 2, 0, 1)
(1, 2, 1, 2, 2, 1, 2)
[1 4 2 2 4 1 1] (4, 2, 4, 1, 2, 4, 1)
(0, 1, 2, 2, 1, 0, 0) (1, 2, 1, 0, 2, 1, 0)
(2, 2, 4, 2, 2, 2, 1)
[1 4 2 2 4 1 1] (4, 2, 2, 1, 1, 4, 1)
(0, 1, 2, 2, 1, 0, 0) (1, 2, 2, 0, 0, 1, 0)
(2, 2, 1, 2, 4, 2, 1)
[(0, 2, 6), (6, 1, 0), (6, 0, 5)]
b'\x00\x02\x06' b'\x00\x02\x00\x02\x02\x00\x02' b'\xf0\xff\xa3C5\xef\x97\x9f\xcfy\xa2\x00\x87GI\xda0T\xfe9\x8b\xb6\xd2\x13}<\x98\xb8-\xf9\x16\x0f' b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x02\x06\x00\x02\x00\x02\x02\x00\x02\xf0\xff\xa3C5\xef\x97\x9f\xcfy\xa2\x00\x87GI\xda0T\xfe9\x8b\xb6\xd2\x13}<\x98\xb8-\xf9\x16\x0f\x00\x00\x00\x00\x00\x00\x00\x00'
b'qp\xd6\xc7\xf8\xb66\xd4y%\x00\xfc\xdd\xdd\x08\xcd\xf0\xff\xa3C5\xef\x97\x9f\xcfy\xa2\x00\x87GI\xda0T\xfe9\x8b\xb6\xd2\x13}<\x98\xb8-\xf9\x16\x0f\x00\x00\x00\x00\x00\x00\x00\x01' b'\xf0\xff\xa3C5\xef\x97\x9f\xcfy\xa2\x00\x87GI\xda0T\xfe9\x8b\xb6\xd2\x13}<\x98\xb8-\xf9\x16\x0f' b'\x00\x00\x00\x00\x0

In [5]:
def setupTree(t):
    max_level = math.log2(t)
    if max_level%1 == 0:
        return 
    max_level = math.ceil(max_level)
    min_level = max_level - 1 
    return

setupTree(11)

4


In [77]:
math.ceil(math.log2(t))

4

In [73]:
2*(12%2**3)

8

In [7]:
def sha256(data):
    return hashlib.sha256(data).digest()

#Prioridad hacia derecha, árboles balanceados a la izquierda
def setupTree(t):
    off = [0]*int(math.ceil(math.log2(t))+1)
    num = []
    levels = math.ceil(math.log2(t))+1
    for i in range(levels):
        num.append(2**i)
    num[-1] = 2*(t%2**(i-1))
    res_nodes = t - 2*(t%2**(i-1))

    off[-2] = res_nodes
    off[-1] = 2**(i-1) - res_nodes
    return off, num

def leafIndices(off,t):
    idx = []
    nodes_i = 2**(math.ceil(math.log2(t))-1)-1
    nodes_last = 2**(math.ceil(math.log2(t))-1) - ((t%2**(math.floor(math.log2(t)))))
    nodes_count = 0
    off_lvl = 0
    for i in range(t):
        if nodes_count == nodes_last:
            off_lvl = off[-1]
        idx.append(nodes_i + off_lvl)
        nodes_i += 1
        nodes_count += 1
    return idx

def updateCtr(ctr, lvl, num):
    lvl_nodes = num[lvl]
    ctr += 2
    if lvl_nodes == ctr:
        lvl -= 1
        ctr = 0
    return ctr, lvl

def merkleRoot(cmt0):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    print(off, num, idx)

    T = [None]*(2*t-1)
    for i in range(0, t):
        T[idx[i]] = cmt0[i]
    print(T)
    lvl = math.floor(math.log2(t)) + 1
    ctr = 0
    for i in range(2*t-2, 0, -2):
        p = math.floor((i-1)/2) + off[lvl-1]
        if '=' in T[i]:
            T[p] = f'd{p}=hash({T[i-1].split("=")[0]}||{T[i].split("=")[0]})'
        else:
            T[p] = f'd{p}=hash({T[i-1]}||{T[i]})'
        ctr, lvl = updateCtr(ctr, lvl, num)
        print(f'i-1:{i-1}, i:{i}, p:{math.floor(p)}, lvl:{lvl}, ctr:{ctr}')
    return T[0], T

t = 7
cmt0 = [f'c{i+1}' for i in range(t)]
d0, T = merkleRoot(cmt0)

print(T)

[0, 0, 1, 3] [1, 2, 4, 6] [3, 7, 8, 9, 10, 11, 12]
[None, None, None, 'c1', None, None, None, 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
i-1:11, i:12, p:6, lvl:3, ctr:2
i-1:9, i:10, p:5, lvl:3, ctr:4
i-1:7, i:8, p:4, lvl:2, ctr:0
i-1:5, i:6, p:2, lvl:2, ctr:2
i-1:3, i:4, p:1, lvl:1, ctr:0
i-1:1, i:2, p:0, lvl:0, ctr:0
['d0=hash(d1||d2)', 'd1=hash(c1||d4)', 'd2=hash(d5||d6)', 'c1', 'd4=hash(c2||c3)', 'd5=hash(c4||c5)', 'd6=hash(c6||c7)', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']


In [5]:
def sha256(data):
    return hashlib.sha256(data).digest()

#Prioridad hacia izquierda, árboles balanceados a la derecha
def setupTree(t):
    off = [0]*int(math.ceil(math.log2(t))+1)
    num = []
    levels = math.ceil(math.log2(t))+1
    for i in range(levels):
        num.append(2**i)
    num[-1] = 2*(t%2**(i-1))
    res_nodes = 2**(i-1) - (t - 2*(t%2**(i-1)))

    i = 0
    for i in [-1,-2]:
        for n in num[:i]:
            off[i] += n
    off[-2] -= res_nodes
    return off, num

def leafIndices(off,t):
    idx = []
    nodes_i = 0 
    nodes_last = (2*(t%2**(math.floor(math.log2(t)))))
    nodes_count = 0
    off_lvl = off[-1]
    for i in range(t):
        if nodes_count == nodes_last:
            off_lvl = off[-2]
        idx.append(nodes_i + off_lvl)
        nodes_i += 1
        nodes_count += 1
    return idx

def updateCtr(ctr, lvl, num):
    lvl_nodes = num[lvl]
    ctr += 2
    if lvl_nodes == ctr:
        lvl -= 1
        ctr = 0
    return ctr, lvl

def merkleRootLeft(cmt0):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    print(off, num, idx)

    T = [None]*(2*t-1)
    for i in range(0, t):
        T[idx[i]] = cmt0[i]
    print(T)
    lvl = math.floor(math.log2(t))+1
    ctr = 0
    for i in range(2*t-2, 0, -2):
        p = math.floor((i-1)/2) # + off[lvl]
        if '=' in T[i-1]:
            T[p] = f'd{p}=hash({T[i-1].split("=")[0]}||{T[i].split("=")[0]})'
        else:
            T[p] = f'd{p}=hash({T[i-1]}||{T[i]})'
        ctr, lvl = updateCtr(ctr, lvl, num)
        print(f'i-1:{i-1}, i:{i}, p:{math.floor(p)}, lvl:{lvl}, ctr:{ctr}')
    return T[0], T

t = 11
cmt0 = [f'c{i+1}' for i in range(t)]
print(merkleRootLeft(cmt0))



[0, 0, 0, 4, 15] [1, 2, 4, 8, 6] [15, 16, 17, 18, 19, 20, 10, 11, 12, 13, 14]
[None, None, None, None, None, None, None, None, None, None, 'c7', 'c8', 'c9', 'c10', 'c11', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6']
i-1:19, i:20, p:9, lvl:4, ctr:2
i-1:17, i:18, p:8, lvl:4, ctr:4
i-1:15, i:16, p:7, lvl:3, ctr:0
i-1:13, i:14, p:6, lvl:3, ctr:2
i-1:11, i:12, p:5, lvl:3, ctr:4
i-1:9, i:10, p:4, lvl:3, ctr:6
i-1:7, i:8, p:3, lvl:2, ctr:0
i-1:5, i:6, p:2, lvl:2, ctr:2
i-1:3, i:4, p:1, lvl:1, ctr:0
i-1:1, i:2, p:0, lvl:0, ctr:0
('d0=hash(d1||d2)', ['d0=hash(d1||d2)', 'd1=hash(d3||d4)', 'd2=hash(d5||d6)', 'd3=hash(d7||d8)', 'd4=hash(d9||c7)', 'd5=hash(c8||c9)', 'd6=hash(c10||c11)', 'd7=hash(c1||c2)', 'd8=hash(c3||c4)', 'd9=hash(c5||c6)', 'c7', 'c8', 'c9', 'c10', 'c11', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6'])


In [9]:
def merkleProofs(T, cmt0, b):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)
    print(idx)
    T_ = [0]*(2*t-1)
    for i in range(0, t):
        if b[i] == 0:
            T_[idx[i]] = 1
    print(T_)

    lvl = math.ceil(math.log2(t))-1
    ctr = 0
    merkle_proofs = []
    for i in range(2*t-2, 0, -2):
        p = math.floor((i-1)/2) + off[lvl]
        T_[p] = T_[i-1] or T_[i]

        print(T_)
        #right child computed, left child not so add it to proof
        if T_[i] == 1 and T_[i-1] == 0:
            merkle_proofs.append(T[i-1])
            print(T[i-1])
        #left child computed, right child not so add it to proof
        if T_[i] == 0 and T_[i-1] == 1:
            merkle_proofs.append(T[i])
            print(T[i])
        print(f'i-1:{i-1}, i:{i}, p:{math.floor(p)}, lvl:{lvl}, ctr:{ctr+2}')
        ctr, lvl = updateCtr(ctr, lvl, num)
    print(T_)

    return merkle_proofs

b = [1,1,1,0,0,0,1]
merkle_proofs = merkleProofs(T, cmt0, b)
print(merkle_proofs)

[3, 7, 8, 9, 10, 11, 12]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0]
c7
i-1:11, i:12, p:6, lvl:2, ctr:2
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
i-1:9, i:10, p:5, lvl:2, ctr:4
[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
i-1:7, i:8, p:3, lvl:1, ctr:2
[0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
i-1:5, i:6, p:2, lvl:0, ctr:2
[0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
i-1:3, i:4, p:1, lvl:0, ctr:4
[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
d1=hash(c1||d4)
i-1:1, i:2, p:0, lvl:0, ctr:6
[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
['c7', 'd1=hash(c1||d4)']


In [199]:
def recomputeMerkleRoot(cmt0, merkle_proofs, b):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    T_ = [0]*(2*t-1)
    T = [0]*(2*t-1)
    for i in range(0, t):
        if b[i] == 0:
            T_[idx[i]] = 1
            T[idx[i]] = cmt0[i]

    lvl = math.ceil(math.log2(t))-1
    ctr = 0
    l = 0
    h = ['hash(0)', 'hash(1)']
    for i in range(2*t-2, 0, -2):
        if T_[i] == 0 and T_[i-1] == 0:
            ctr, lvl = updateCtr(ctr, lvl, num)
            continue
    
        #take first node from tree if valid, else from proof
        if T_[i] == 1:
            h[1] = T[i]
        else:
            h[1] = merkle_proofs[l]
            l += 1
        
        #take second node from tree if valid, else from proof
        if T_[i-1] == 1:
            h[0] = T[i-1]
        else:
            h[0] = merkle_proofs[l]
            l += 1
        
        p = math.floor((i-1)/2) + off[lvl]
        if '=' in h[0] or '=' in h[1]:
            T[p] = f'd{p}=hash({h[0].split("=")[0]}||{h[1].split("=")[0]})'
        else:
            T[p] = f'd{p}=hash({h[0]}||{h[1]})'
        T_[p] = 1

        print(f'h:{h},Ti-1:{T_[i-1]}, Ti{T_[i]}')
        print(f'i-1:{i-1}, i:{i}, p:{math.floor(p)}, lvl:{lvl}, ctr:{ctr+2}')
        print(f'T:{T}\nT_:{T_}')
        ctr, lvl = updateCtr(ctr, lvl, num)

    d_0 = T[0]
    return d_0

print(recomputeMerkleRoot(cmt0, merkle_proofs, b))

h:['c6', 'c7'],Ti-1:1, Ti0
i-1:11, i:12, p:6, lvl:2, ctr:2
T:[0, 0, 0, 0, 0, 0, 'd6=hash(c6||c7)', 0, 0, 'c4', 'c5', 'c6', 0]
T_:[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0]
h:['c4', 'c5'],Ti-1:1, Ti1
i-1:9, i:10, p:5, lvl:2, ctr:4
T:[0, 0, 0, 0, 0, 'd5=hash(c4||c5)', 'd6=hash(c6||c7)', 0, 0, 'c4', 'c5', 'c6', 0]
T_:[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
h:['d5=hash(c4||c5)', 'd6=hash(c6||c7)'],Ti-1:1, Ti1
i-1:5, i:6, p:2, lvl:0, ctr:2
T:[0, 0, 'd2=hash(d5||d6)', 0, 0, 'd5=hash(c4||c5)', 'd6=hash(c6||c7)', 0, 0, 'c4', 'c5', 'c6', 0]
T_:[0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
h:['d1=hash(c1||d4)', 'd2=hash(d5||d6)'],Ti-1:0, Ti1
i-1:1, i:2, p:0, lvl:0, ctr:6
T:['d0=hash(d1||d2)', 0, 'd2=hash(d5||d6)', 0, 0, 'd5=hash(c4||c5)', 'd6=hash(c6||c7)', 0, 0, 'c4', 'c5', 'c6', 0]
T_:[1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
d0=hash(d1||d2)


In [7]:
import numpy as np
gamma = np.array([1,6,4,3,2,1,5])
u = np.array([0,3,0,5,1,0,2])
s = np.array([1,0,4,0,0,5,6])
u_as = np.array([2,3,1,5,1,3,0])
sigma = [3,5,4,6,1,0,2]
u_ = []
gamma_ = [] 
s_ = []
u_as_ = []
for i in sigma:
    u_.append(u[i])
    gamma_.append(gamma[i])
    s_.append(s[i])
    u_as_.append(u_as[i])
print('Op(u)=',np.multiply(gamma_,u_))
print('Op(s)=',np.multiply(gamma_,s_))
print('Op(u+as)=',np.multiply(gamma_,u_as_))

Op(u)= [15  0  2 10 18  0  0]
Op(s)= [ 0  5  0 30  0  1 16]
Op(u+as)= [15  3  2  0 18  2  4]


In [ ]:
def generate_v_matrix(seed_pk, rows, cols, p):
    # Inicializamos la matriz de tamaño rows x cols
    V = []
    max_bytes = 8160  # Límite máximo de HKDF en bytes
    max_bits = max_bytes * 8  # Límite máximo en bits
    
    # Calculamos el número de bits por bloque de la matriz
    bits_per_entry = math.ceil(math.log2(p))  # Bits por entrada en F_p
    entries_per_block = max_bits // bits_per_entry  # Máximo de entradas por bloque
    
    # Generamos los bloques iterativamente
    total_entries = rows * cols
    total_blocks = math.ceil(total_entries / entries_per_block)  # Total de bloques necesarios
    
    for block_index in range(total_blocks):
        # Usamos HKDF con el mismo seed pero con un prefijo único para cada bloque
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=max_bytes,
            salt=None,
            info=b"CROSS-R-SDP matrix block " + block_index.to_bytes(4, byteorder="big"),
            backend=default_backend()
        )
        random_bits = hkdf.derive(seed_pk)
        
        # Convertimos los bits a valores en F_p
        block_entries = []
        for i in range(entries_per_block):
            if len(block_entries) + len(V) >= total_entries:
                break  # Terminamos si ya llenamos la matriz
            # Extraemos los bits necesarios para cada entrada
            bit_offset = i * bits_per_entry
            entry_bits = random_bits[bit_offset // 8:(bit_offset + bits_per_entry + 7) // 8]
            entry_int = int.from_bytes(entry_bits, byteorder="big") % p
            block_entries.append(entry_int)
        
        # Añadimos las entradas del bloque a la matriz
        V.extend(block_entries)
    
    # Convertimos la lista de entradas a una matriz de Sage
    V_matrix = matrix(GF(p), rows, cols, V)
    return V_matrix

In [ ]:
def generate_eta_vector(seede, n, z):
    """
    Genera un vector η ∈ F_z^n usando CSPRNG(Seede, F_z^n).
    
    Parámetros:
    - seede: Semilla para la generación segura.
    - n: Longitud del vector (número de elementos).
    - z: Primer primo mayor a 2^λ (orden del campo finito).
    
    Retorna:
    - Un vector η en F_z^n.
    """
    max_bytes = 8160  # Límite máximo de HKDF en bytes
    max_bits = max_bytes * 8  # Límite máximo en bits
    
    # Calculamos el número de bits por elemento en F_z
    bits_per_entry = math.ceil(math.log2(z))
    total_bits = n * bits_per_entry  # Total de bits necesarios para el vector completo
    
    if total_bits <= max_bits:
        # Si el vector cabe en una única llamada a HKDF
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=(total_bits + 7) // 8,  # Convertimos de bits a bytes
            salt=None,
            info=b"CROSS-R-SDP vector generation",
            backend=default_backend()
        )
        random_bits = hkdf.derive(seede)
    else:
        # Si el vector excede el límite de HKDF, generamos en bloques
        random_bits = bytearray()
        blocks_needed = math.ceil(total_bits / max_bits)
        for block_index in range(blocks_needed):
            hkdf = HKDF(
                algorithm=hashes.SHA256(),
                length=max_bytes,
                salt=None,
                info=b"CROSS-R-SDP vector block " + block_index.to_bytes(4, byteorder="big"),
                backend=default_backend()
            )
            random_bits.extend(hkdf.derive(seede))
    
    # Convertimos los bits en elementos en F_z
    vector_entries = []
    for i in range(n):
        bit_offset = i * bits_per_entry
        entry_bits = random_bits[bit_offset // 8:(bit_offset + bits_per_entry + 7) // 8]
        entry_int = int.from_bytes(entry_bits, byteorder="big") % z
        vector_entries.append(entry_int)
    
    # Convertimos a un vector en Sage
    eta_vector = vector(GF(z), vector_entries)
    return eta_vector

In [ ]:
def generar_elementos_orden_z(lambda_security):
    # Parámetros básicos
    p = next_prime(2 ** lambda_security)  # Seleccionar un primo suficientemente grande
    z = next_prime(2 ** (lambda_security // 2))  # Orden del subgrupo
    Fp = GF(p)  # Campo finito Fp
    
    # Encontrar un generador del grupo multiplicativo Fp^*
    g = Fp.multiplicative_generator()  # Generador de Fp^*
    
    # Generar todos los elementos de orden z
    elementos_orden_z = []
    
    for k in range(1, p):
        if gcd(k, z) == 1:  # Si k es coprimo con z
            candidato = g^k  # g^k es un elemento de Fp
            if candidato.multiplicative_order() == z:
                elementos_orden_z.append(candidato)
    
    return elementos_orden_z

In [ ]:
def keygen_cross_r_sdp(lambda_security):
    """
    Genera las claves públicas y privadas para CROSS-R-SDP utilizando semillas para minimizar el almacenamiento.
    
    Parámetros:
        lambda_security (int): Parámetro de seguridad del esquema (por ejemplo, 128, 192, 256).
        
    Retorna:
        dict: Diccionario con las claves públicas y privadas.
    """
    # 1. Generar la semilla principal Seed_sk desde un TRNG
    seed_sk = secrets.token_bytes(lambda_security // 8)  # Semilla de seguridad en bytes
    print("Seed sk generada")
    
    # 2. Derivar Seed_e y Seed_pk desde Seed_sk
    #seed_e = hashlib.sha256(seed_sk + b'e').digest()  # Derivación determinística
    #seed_pk = hashlib.sha256(seed_sk + b'pk').digest()
    
    # 2. Usar un CSPRNG (HKDF) para derivar Seed_e y Seed_pk desde Seed_sk
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=2 * (lambda_security // 8),  # Generar 2 semillas de tamaño λ cada una
        salt=None,  # Sin sal adicional, para simplificar
        info=b"CROSS-R-SDP key derivation",
        backend=default_backend()
    )
    derived_keys = hkdf.derive(seed_sk)
    seed_e = derived_keys[:lambda_security // 8]  # Primera mitad para Seed_e
    seed_pk = derived_keys[lambda_security // 8:]  # Segunda mitad para Seed_pk
    print("Seed e y seed pk generadas con CSPRNG")
    
    # 3-prev. Generar parámetros básicos 
    p = next_prime(2 ** lambda_security)  # Primo grande
    #z = next_prime(2 ** (lambda_security // 2))  # Orden del subgrupo
    Fp = GF(p)  # Campo finito Fp
    g = Fp.random_element()  # Generador aleatorio
    P = Primes()
    while g.multiplicative_order() < 2 ** (lambda_security // 2) or g.multiplicative_order() not in P:  # Asegurar que g tiene orden z
        g = Fp.random_element()
    z = g.multiplicative_order()
    n = lambda_security * 2
    k = lambda_security

    # 3. Generar matriz V desde Seed_pk
    V = generate_v_matrix(seed_pk, n-k, k, p)
    print("Matriz V generada")

    # 4. Construir matriz H usando V
    I_nk = identity_matrix(Fp, (n-k))  # Matriz identidad de tamaño k
    H = block_matrix([[I_nk, V]])  # H es una matriz (n-k) x n
    print("Matriz H generada")

    # 5. Generar un vector η ∈ F_z^n usando CSPRNG(Seede, F_z^n).
    eta = generate_eta_vector(seed_e, n, z)
    print("Vector η generado")

    # 6-8. Obtener el valor de e
    e = []
    for j in range(n):
        e.append(g**(eta[j])%p)
    e = vector(e)
    print("Vector e calculado")

    # 9. Obtener el valor de s
    #print(e)
    #print(H)
    s = e*H.transpose()
    print("Síndrome s calculado")

    # 10-12. Construir claves pública y privada
    public_key = {
        "Seed_pk": seed_pk,
        "s": s
    }
    private_key = {
        "Seed_sk": seed_sk
    }

    return {"public_key": public_key, "private_key": private_key}
    '''
    g = Fp.random_element()  # Generador aleatorio
    while g.multiplicative_order() != z:  # Asegurar que g tiene orden z
        g = Fp.random_element()
    
    # Generar el subgrupo G
    G = [g ** i for i in range(1, z + 1)]

    # 4. Generar las matrices H y MG desde Seed_pk
    random_gen = Random(seed_pk)  # Usar la semilla para generar las matrices
    n = lambda_security * 2  # Número de columnas
    k = lambda_security      # Dimensiones de H: (n-k) x n
    m = lambda_security      # Dimensión de MG: m x n
    H = random_matrix(Fp, n - k, n, algorithm="random", seed=random_gen.random())  # Matriz H
    MG = random_matrix(Fp, m, n, algorithm="random", seed=random_gen.random())  # Matriz MG

    # 5. Generar vector restringido ζ y calcular η y e
    random_gen_e = Random(seed_e)  # Usar Seed_e para generar ζ
    zeta = vector([random_gen_e.randint(0, z - 1) for _ in range(m)])  # ζ en Fz^m
    eta = zeta * MG  # η = ζMG en Fz^n
    e = vector([g ** eta[i] for i in range(n)])  # e = (g^η[0], ..., g^η[n-1]) ∈ G

    # 6. Calcular síndrome s
    s = e * H.transpose()

    # 7. Construir claves pública y privada
    public_key = {
        "Seed_pk": seed_pk,
        "s": s
    }
    private_key = {
        "Seed_sk": seed_sk
    }

    return {"public_key": public_key, "private_key": private_key}
    '''

In [ ]:
keys = keygen_cross_r_sdp(64)
print("Clave pública:", keys["public_key"])
print("Clave privada:", keys["private_key"])

Seed sk generada
Seed e y seed pk generadas con CSPRNG
Matriz V generada
Matriz H generada
Vector η generado
Vector e calculado
Síndrome s calculado
Clave pública: {'Seed_pk': b'o8\xa3B\xd4 {F', 's': (12689790102570935737, 17290422669752356626, 12447982102400172154, 7368968899834045008, 9692515634713675269, 3857248561948286162, 11549586440679395454, 13944926540037478504, 970289067832606035, 2274367561721997735, 1668254968362353134, 6686708400489843845, 5627393177923768862, 5038370417575172785, 2633299530359835603, 8101012040047375236, 8468750386724245053, 3542775231180811808, 4348852526592382897, 18129474856368290201, 10898526699507651064, 1517876694454929757, 11001233693573680652, 2061791100656646131, 13911995396826139834, 5441843169351282262, 3612260834608250187, 14870658969047766470, 10841619840631740162, 5728045967264528314, 5229821135263022056, 9466018978737175262, 18023237039403308599, 7275705702764941191, 11509457893083509436, 3397971458268893269, 9004891019243052546, 1766152764

In [ ]:
def sign_cross_r_sdp(private_key, msg):
    """
    Genera las claves públicas y privadas para CROSS-R-SDP utilizando semillas para minimizar el almacenamiento.
    
    Parámetros:
        lambda_security (int): Parámetro de seguridad del esquema (por ejemplo, 128, 192, 256).
        
    Retorna:
        dict: Diccionario con las claves públicas y privadas.
    """
    # 1. Expandir la semilla privada Seed_sk
    
    # 1.1. Usar un CSPRNG (HKDF) para derivar Seed_e y Seed_pk desde Seed_sk
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=2 * (lambda_security // 8),  # Generar 2 semillas de tamaño λ cada una
        salt=None,  # Sin sal adicional, para simplificar
        info=b"CROSS-R-SDP key derivation",
        backend=default_backend()
    )
    derived_keys = hkdf.derive(seed_sk)
    seed_e = derived_keys[:lambda_security // 8]  # Primera mitad para Seed_e
    seed_pk = derived_keys[lambda_security // 8:]  # Segunda mitad para Seed_pk
    print("Seed e y seed pk generadas con CSPRNG")

    # 1.2-prev. Generar parámetros básicos 
    p = next_prime(2 ** lambda_security)  # Primo grande
    #z = next_prime(2 ** (lambda_security // 2))  # Orden del subgrupo
    Fp = GF(p)  # Campo finito Fp
    g = Fp.random_element()  # Generador aleatorio
    P = Primes()
    while g.multiplicative_order() < 2 ** (lambda_security // 2) or g.multiplicative_order() not in P:  # Asegurar que g tiene orden z
        g = Fp.random_element()
    z = g.multiplicative_order()
    n = lambda_security * 2
    k = lambda_security

    # 1.2. Generar matriz V desde Seed_pk
    V = generate_v_matrix(seed_pk, n-k, k, p)
    print("Matriz V generada")

    # 1.3. Construir matriz H usando V
    I_nk = identity_matrix(Fp, (n-k))  # Matriz identidad de tamaño k
    H = block_matrix([[I_nk, V]])  # H es una matriz (n-k) x n
    print("Matriz H generada")

    # 1.4. Generar un vector η ∈ F_z^n usando CSPRNG(Seede, F_z^n).
    eta = generate_eta_vector(seed_e, n, z)
    print("Vector η generado")

    # 2. Generar MSeed y Salt con tamaño λ y 2λ bits (2 * λ / 8 bytes) respectivamente.
    mseed = secrets.token_bytes(lambda_security // 8) 
    salt = secrets.token_bytes(2 * lambda_security // 8)

    

In [ ]:
#Funciones auxiliares para el algoritmo

def CSPRNG(seed, output_size):
    """Función para simular un generador criptográficamente seguro (CSPRNG)."""
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=output_size,
        salt=None,
        info=b'',
    )
    return hkdf.derive(seed)

def hash_function(data):
    """Función hash para computar el valor hash."""
    digest = hashes.Hash(hashes.SHA256())
    digest.update(data)
    return digest.finalize()

def calculate_merkle_root(cmt0):
    """
    Calcula la raíz de un árbol de Merkle a partir de los compromisos en cmt0.

    Parámetros:
        cmt0 (list): Lista de compromisos que serán las hojas del árbol.

    Retorna:
        bytes: Raíz del árbol de Merkle como un hash.
    """
    # Asegurar que hay al menos un compromiso
    if len(cmt0) == 0:
        raise ValueError("La lista cmt0 no puede estar vacía.")
    
    # Convertir los compromisos a una lista de hashes (si no lo están ya)
    current_level = [hash_function(cmt) if not isinstance(cmt, bytes) else cmt for cmt in cmt0]
    
    # Construir el árbol de Merkle
    while len(current_level) > 1:
        next_level = []
        # Combinar nodos en pares
        for i in range(0, len(current_level), 2):
            if i + 1 < len(current_level):  # Si hay un par
                combined = current_level[i] + current_level[i + 1]
            else:  # Nodo impar, combinar consigo mismo
                combined = current_level[i] + current_level[i]
            # Calcular el hash del par combinado
            next_level.append(hash_function(combined))
        current_level = next_level  # Subir al siguiente nivel

    # La raíz es el único elemento restante
    return current_level[0]


# Convertir entero a bytes
def int_to_bytes(value, length=4):
    return value.to_bytes(length, byteorder='big')

# Concatenar múltiples byte arrays
def concatenate(*args):
    return b''.join(args)

def vector_to_bytes(vector):
    return b''.join(int(x).to_bytes((int(x).bit_length() + 7) // 8, 'big') for x in vector)

# Crear árbol binario y generar las hojas
def seed_tree_leaves(t, Mseed, Salt, lambda_bits):
    leaves = []
    queue = deque()

    # Concatenar Mseed y Salt para formar la raíz
    root = concatenate(Mseed, Salt)
    queue.append(root)

    node_index = 0  # Índice del nodo
    while len(leaves) < t:
        parent = queue.popleft()  # Obtener el nodo padre

        # Generar nodos hijo a partir del nodo padre
        for _ in range(2):  # Dos hijos por nodo (árbol binario)
            node_data = generate_node_data(parent, Salt, node_index, lambda_bits)
            queue.append(node_data)

            # Verificar si el nodo es hoja
            if is_leaf(node_data):
                leaves.append(node_data)
                if len(leaves) == t:
                    break  # Si se completan t hojas, detener

            node_index += 1

    return leaves  # Retornar las hojas como las semillas

# Generar datos de un nodo a partir del nodo padre, Salt, e índice
def generate_node_data(parent, Salt, index, lambda_bits):
    index_bytes = int_to_bytes(index)  # Convertir índice a bytes
    data = concatenate(parent, Salt, index_bytes)  # Concatenar datos
    return CSPRNG(data, 3 * lambda_bits // 8)  # Generar 3λ bits

# Determinar si un nodo es hoja (basado en una condición personalizada)
def is_leaf(node):
    # Para este caso, todos los nodos se consideran hojas
    return True

def generate_merkle_proofs(cmt0, b):
    """
    Genera las pruebas de Merkle para los índices de compromiso necesarios.

    Args:
        cmt0 (list): Secuencia de compromisos.
        b (list): Vector de desafío fijo.

    Returns:
        list: Estructura de MerkleProofs con los nodos relevantes.
    """
    from hashlib import sha256

    def hash_node(data):
        return sha256(data).digest()

    # Construcción del árbol de Merkle
    def build_merkle_tree(leaves):
        tree = [leaves]
        while len(tree[-1]) > 1:
            layer = []
            for i in range(0, len(tree[-1]), 2):
                left = tree[-1][i]
                right = tree[-1][i + 1] if i + 1 < len(tree[-1]) else left
                layer.append(hash_node(left + right))
            tree.append(layer)
        return tree

    # Construir árbol de Merkle
    leaf_hashes = [hash_node(leaf) for leaf in cmt0]
    tree = build_merkle_tree(leaf_hashes)

    # Generar pruebas relevantes
    merkle_proofs = []
    for i, bit in enumerate(b):
        if bit == 1:  # Solo incluir pruebas para b[i] = 1
            proof = []
            idx = i
            for layer in tree[:-1]:
                sibling_idx = idx ^ 1  # Nodo hermano
                if sibling_idx < len(layer):
                    proof.append(layer[sibling_idx])
                idx //= 2
            merkle_proofs.append(proof)

    return merkle_proofs

def generate_seed_tree_paths(Mseed, b, t, lambda_security):
    """
    Genera las rutas del árbol de semillas para los desafíos indicados por b.

    Args:
        Mseed (bytes): Semilla maestra.
        b (list): Vector de desafío fijo.
        t (int): Número de rondas.
        lambda_security (int): Parámetro de seguridad.

    Returns:
        list: Estructura SeedPaths con las rutas relevantes.
    """
    from hashlib import sha256

    def derive_seed(seed, index):
        return sha256(seed + index.to_bytes((index.bit_length() + 7) // 8, 'big')).digest()

    # Construcción del árbol de semillas
    seed_tree = [Mseed]
    for i in range(t):
        seed_tree.append(derive_seed(seed_tree[-1], i))

    # Selección de nodos relevantes
    seed_paths = []
    for i, bit in enumerate(b):
        if bit == 1:  # Solo incluir nodos relevantes para b[i] = 1
            node = seed_tree[i]
            seed_paths.append(node)

    return seed_paths




In [ ]:
def sign_cross_r_sdp(private_key, msg, lambda_security=128, t=10):# t, n, lambda_security, Fp, Fn_z):
    """
    Algoritmo Sign hasta la línea 16.

    Parámetros:
        private_seed (bytes): Semilla privada.
        t (int): Número de rondas del protocolo.
        n (int): Tamaño de los vectores.
        λ (int): Parámetro de seguridad (bits).
        Fp (int): Campo primo para operaciones en Fp.
        Fn_z (int): Campo primo para operaciones en Fn_z.

    Retorna:
        tuple: cmt0 y cmt1 generados.
    """
    # 1. Expandir la semilla privada Seed_sk
    
    # 1.1. Usar un CSPRNG (HKDF) para derivar Seed_e y Seed_pk desde Seed_sk
    seed_sk = private_key["Seed_sk"]
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=2 * (lambda_security // 8),  # Generar 2 semillas de tamaño λ cada una
        salt=None,  # Sin sal adicional, para simplificar
        info=b"CROSS-R-SDP key derivation",
        backend=default_backend()
    )
    derived_keys = hkdf.derive(seed_sk)
    seed_e = derived_keys[:lambda_security // 8]  # Primera mitad para Seed_e
    seed_pk = derived_keys[lambda_security // 8:]  # Segunda mitad para Seed_pk
    print("Seed e y seed pk generadas con CSPRNG")

    # 1.2-prev. Generar parámetros básicos 
    p = next_prime(2 ** lambda_security)  # Primo grande
    #z = next_prime(2 ** (lambda_security // 2))  # Orden del subgrupo
    Fp = GF(p)  # Campo finito Fp
    g = Fp.random_element()  # Generador aleatorio
    P = Primes()
    while g.multiplicative_order() < 2 ** (lambda_security // 2) or g.multiplicative_order() not in P:  # Asegurar que g tiene orden z
        g = Fp.random_element()
    z = g.multiplicative_order()
    n = lambda_security * 2
    k = lambda_security

    # 1.2. Generar matriz V desde Seed_pk
    V = generate_v_matrix(seed_pk, n-k, k, p)
    print("Matriz V generada")

    # 1.3. Construir matriz H usando V
    I_nk = identity_matrix(Fp, (n-k))  # Matriz identidad de tamaño k
    H = block_matrix([[I_nk, V]])  # H es una matriz (n-k) x n
    print("Matriz H generada")

    # 1.4. Generar un vector η ∈ F_z^n usando CSPRNG(Seede, F_z^n).
    eta = generate_eta_vector(seed_e, n, z)
    print("Vector η generado")

    # 2. Generar Mseed y Salt
    Mseed = secrets.token_bytes(lambda_security // 8)  # λ bits aleatorios
    Salt = secrets.token_bytes( (2*lambda_security) // 8)   # 2λ bits aleatorios
    print("Mseed y Salt generados")

    # 3. Expandir las semillas para las rondas
    Seed_tree = seed_tree_leaves(t, Mseed, Salt, lambda_security)
    print("Seed tree generado")

    # Inicializar listas de compromisos
    cmt0 = []
    cmt1 = []

    # 4. Bucle para calcular compromisos
    for i in range(t):
        print("Calculado cmt0 y cmt1 de la iteración "+str(i+1) +".")
        # 5. Expandir Seed[i] en Seedu′ y Seede′
        combined_seed = CSPRNG(Seed_tree[i], 2 * (lambda_security // 8))
        seedu_prime, seede_prime = combined_seed[:lambda_security // 8], combined_seed[lambda_security // 8:]

        # 6. Generar η′_i
        eta_prime_i = generate_eta_vector(seede_prime, n, z)  # η′_i como vector aleatorio

        # 7. Calcular σ_i
        sigma_i = (eta - eta_prime_i) #% p

        # 8. Calcular v[j]
        v = [0]*n
        for j in range(n):
            v[j] = g**(sigma_i[j])
        v = vector(v)
        #v = np.array([pow(g, sigma_i[j], Fp) for j, g in enumerate(np.random.randint(0, Fp, size=n))])  # g^(σ_i[j])

        # 11. Expandir u′_i
        u_prime_i = generate_eta_vector(seedu_prime, n, p)  # u′_i como vector aleatorio

        # 12. Calcular u = v ⋆ u′_i
        u = v.pairwise_product(u_prime_i) #% p

        # 13. Calcular síndrome s^˜ = uH^⊤
        s_t = u*H.transpose()

        # 14. Calcular cmt0[i]
        cmt0_i = hash_function(
            b''.join([
                vector_to_bytes(s_t),
                vector_to_bytes(sigma_i),
                Salt,
                i.to_bytes((i.bit_length() + 7) // 8, 'big')
            ])
        )
        cmt0.append(cmt0_i)

        # 15. Calcular cmt1[i]
        cmt1_i = hash_function(
            b''.join([
                Seed_tree[i],
                Salt,
                i.to_bytes((i.bit_length() + 7) // 8, 'big')
            ])
        )
        cmt1.append(cmt1_i)

    print("Cmt0 y Cmt1 generados")
    
    # 17. Calcular d0 como MerkleRoot de cmt0[0], ..., cmt0[t-1]
    d0 = calculate_merkle_root(cmt0)
    print("MerkleRoot d0 calculado")

    # 18. Calcular d1 como Hash de la concatenación de todos los cmt1
    d1 = hash_function(b''.join(cmt1))
    print("d1 calculado")

    # 19. Calcular d01 como Hash(d0 || d1)
    d01 = hash_function(d0 + d1)
    print("d01 calculado")

    # 20. Calcular dm como Hash del mensaje
    dm = hash_function(msg.encode('utf-8'))
    print("dm calculado")

    # 21. Calcular dβ como Hash(dm || d01 || Salt)
    d_beta = hash_function(dm + d01 + Salt)
    print("dβ calculado")

    # 22. Generar beta usando CSPRNG y (F^*_p)^t
    beta = CSPRNG(d_beta, t)
    print("beta calculado")
    print("Extracción del primer vector reto completado")

    # Inicializar lista de respuestas
    y = []

    # 23. Calcular las respuestas en el primer bucle
    for i in range(t):
        # Inicializar el vector e' para esta iteración
        e_prime = []
        
        # 24. Iterar sobre cada componente del vector para calcular e'
        for j in range(n):
            # 25. Calcular e'[j] como g^(η′_i[j])
            e_prime.append(g ** eta_prime_i[j])
        
        # 27. Calcular y_i como u'_i + beta[i] * e'_i
        beta_i_e_prime = vector([beta[i] * e_prime[j] for j in range(n)])  # Escalar por componente
        y_i = u_prime_i + beta_i_e_prime
        
        # Agregar y_i a la lista de respuestas
        y.append(y_i)
    print("Cálculo del primer round de respuestas completado")

    # 29. db ← Hash(y0 || ... || yt−1 || dβ)
    db = hash_function(b''.join([vector_to_bytes(y_i) for y_i in y]) + d_beta)
    print("db calculado")

    # 30. b ← CSPRNG(db, B^t_w)
    b = CSPRNG(db, t)  # Generar vector de peso w con el CSPRNG
    print("b calculado")
    print("Extración del segundo vector reto completado")

    # 31. MerkleProofs ← MerkleProof((cmt0[0], ..., cmt0[t-1]), b)
    MerkleProofs = generate_merkle_proofs(cmt0, b)
    print("Pruebas de Merkle generadas")
    print(cmt0, b)

    # 32. SeedPath ← SeedTreePaths(Mseed, b)
    SeedPath = generate_seed_tree_paths(Mseed, b, t, lambda_security)
    print("Rutas del árbol de semillas generadas")

    # Inicializar listas rsp0 y rsp1
    rsp0 = []  # Respuestas para los índices donde b[i] = 0
    rsp1 = []  # Respuestas para compromisos donde b[i] = 0

    # 35. Inicializar índice j
    j = 0

    # 36. Bucle para calcular rsp0 y rsp1
    for i in range(t):
        if b[i] == 0:
            # 38. rsp0[j] ← (yi, σi)
            rsp0.append((y[i], cmt0[i]))

            # 39. rsp1[j] ← cmt1[i]
            rsp1.append(cmt1[i])

            # Incrementar j
            j += 1

    print("Respuestas rsp0 y rsp1 calculadas")

    # 43. Componer la firma
    Signature_show = {
        "Salt": Salt,
        "d01": d01,
        "db": db,
        "MerkleProofs": MerkleProofs,
        "SeedPath": SeedPath,
        "rsp0": rsp0,
        "rsp1": rsp1,
    }
    print("Firma generada\n",Signature_show)
    Signature = (
    Salt.encode('utf-8') +
    private_key["d01"].encode('utf-8') +
    db.encode('utf-8') +
    MerkleProofs.encode('utf-8') +
    SeedPath.encode('utf-8') +
    rsp0.encode('utf-8') +
    rsp1.encode('utf-8')
    )
    print("Firma convertida a bytestring correctamente.")

    # 44. Retornar la firma
    return Signature



    

In [ ]:
sign_cross_r_sdp(keys["private_key"], "Hola Mundo", 64)

Seed e y seed pk generadas con CSPRNG
Matriz V generada
Matriz H generada
Vector η generado
Mseed y Salt generados
Seed tree generado
Calculado cmt0 y cmt1 de la iteración 1.
Calculado cmt0 y cmt1 de la iteración 2.
Calculado cmt0 y cmt1 de la iteración 3.
Calculado cmt0 y cmt1 de la iteración 4.
Calculado cmt0 y cmt1 de la iteración 5.
Calculado cmt0 y cmt1 de la iteración 6.
Calculado cmt0 y cmt1 de la iteración 7.
Calculado cmt0 y cmt1 de la iteración 8.
Calculado cmt0 y cmt1 de la iteración 9.
Calculado cmt0 y cmt1 de la iteración 10.
Cmt0 y Cmt1 generados
MerkleRoot d0 calculado
d1 calculado
d01 calculado
dm calculado
dβ calculado
beta calculado
Extracción del primer vector reto completado
Cálculo del primer round de respuestas completado
db calculado
b calculado
Extración del segundo vector reto completado
Pruebas de Merkle generadas
[b'W\xdb!O z\xbd\x9f\x8f|\x84=Osv\xa6x\xef\x19O\xe6\xa5+\x85\xe4\xb1\\\xb9z>\x87\xf4', b'\x13ty\xeb1\x89\xcdE\xb9/{\xfeQ\xc1\xb5\x1c\xff\xa8\x0e\\\x

AttributeError: 'bytes' object has no attribute 'encode'